In [1]:
!pip install langchain langchain_google_genai PyPDF2 faiss-cpu langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.7 MB/s eta 0:00:00


In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.output_parsers import StructuredOutputParser
import os
from IPython.display import display, Markdown
import re
from io import BytesIO
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [3]:
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [4]:
def get_pdf_text(pdf_files):
    text = ""
    for i,uploaded_file in enumerate(pdf_files):
        # Convert the uploaded file to a BytesIO stream
        pdf_stream = BytesIO(uploaded_file.read())
        pdf_reader = PdfReader(pdf_stream)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
    return text

In [27]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000,chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

In [14]:
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    vector_store = FAISS.from_texts(text_chunks,embedding=embeddings)
    vector_store.save_local('faiss_index')

In [24]:
def get_conversational_chain():
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer
    If there is 4 numerical columns in table, first 2 for months ended 31 march, next 2 for year ended march 31
    If there is 2 numerical columns then those are march 31 2024, march 31 2023 in a row. These are for numerical columns only.
    Context:\n{context}?\n
    Question:\n{question}\n
    Answer:
    """

    model = ChatGoogleGenerativeAI(model='gemini-1.5-flash',temparature=0.3)
    prompt = PromptTemplate(template=prompt_template,input_variables=["context",'question'])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

In [25]:
def user_input(user_question):
    # Ensure embeddings match the creation process
    embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

    # Load the FAISS index with dangerous deserialization explicitly allowed
    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question, k=1)

    chain = get_conversational_chain()

    response = chain(
        {"input_documents": docs, 'question': user_question},
        return_only_outputs=True)

    print(response["output_text"])
    display(Markdown("## Supporting Documents: "))
    for doc in docs:
        model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.7)
        prompt =f"""
        Extract tables from {doc.page_content}. Don't give any information other than the tables.
        If there is 4 numerical columns in table, first 2 for months ended 31 march, next 2 for year ended march 31
        If there is 2 numerical columns then those are march 31 2024, march 31 2023 in a row. These are for numerical columns only.
        """
        response = model.invoke(f"Extract tables from {doc.page_content}. Don't give any information other than the tables.")
        display(Markdown(response.content))

In [28]:
path = "/content/Sample Financial Statement.pdf"
pdf_files = [open(path, "rb")]
text = get_pdf_text(pdf_files)
text_chunks = get_text_chunks(text)
get_vector_store(text_chunks)

In [29]:
user_question = "compare the travel expenses of 2024 and 2023 year end"
user_input(user_question)

The travel expenses for the year ended March 31, 2024 were ₹1,759 crore, while for the year ended March 31, 2023 they were ₹1,525 crore.


## Supporting Documents: 

**Table 1: Income Statement (₹ crore)**

| Item                                      | Year ended March 31, 2023 | Three months ended March 31, 2024 | Year ended March 31, 2024 | Three months ended March 31, 2023 |
|-------------------------------------------|----------------------------|---------------------------------|----------------------------|---------------------------------|
| Travel expenses                           | 471                         | 426                              | 1,759                         | 1,525                              |
| Cost of software packages and others      | 3,687                        | 2,886                             | 13,515                        | 10,902                             |
| Communication expenses                     | 147                         | 171                              | 677                          | 713                               |
| Consultancy and professional charges      | 489                         | 387                              | 1,726                         | 1,684                              |
| Depreciation and amortization expenses    | 1,163                        | 1,121                             | 4,678                         | 4,225                              |
| Finance cost                              | 110                          | 82                               | 470                          | 284                               |
| Other expenses                            | 985                          | 1,146                             | 4,716                         | 4,392                              |
| **Total expenses**                        | **30,412**                    | **29,646**                         | **122,393**                    | **116,146**                        |
| Profit before tax                         | 10,240                       | 8,466                             | 35,988                       | 33,322                             |
| Current tax                               | 1,173                        | 2,260                             | 8,390                         | 9,287                              |
| Deferred tax                              | 1,092                        | 72                               | 1,350                         | (73)                               |
| **Profit for the period**                 | **7,975**                     | **6,134**                          | **26,248**                    | **24,108**                         |
| Remeasurement of the net defined benefit liability/asset, net | 26                          | 25                               | 120                          | 8                                 |
| Equity instruments through other comprehensive income, net | (12)                         | (15)                             | 19                           | (7)                                |
| Fair value changes on derivatives designated as cash flow hedge, net | 28                          | 36                               | 11                           | (7)                                |
| Fair value changes on investments, net    | 37                          | 42                               | 144                          | (256)                              |
| **Total other comprehensive income /(loss), net of tax** | **(152)**                    | **149**                           | **520**                     | **514**                            |
| **Total comprehensive income for the period** | **7,823**                     | **6,283**                          | **26,768**                    | **24,622**                         |
| Profit attributable to: Owners of the Company | 7,969                        | 6,128                             | 26,233                        | 24,095                             |
| Profit attributable to: Non-controlling interests | 6                           | 6                               | 15                           | 13                                |
| **Total comprehensive income attributable to: Owners of the Company** | **7,821**                     | **6,276**                          | **26,754**                    | **24,598**                         |
| **Total comprehensive income attributable to: Non-controlling interests** | **2**                          | **7**                              | **14**                         | **24**                              |
| Basic (in ₹ per share)                   | 19.25                        | 14.79                             | 63.39                         | 57.63                              |
| Diluted (in ₹ per share)                 | 19.22                        | 14.77                             | 63.29                         | 57.54                              |


**Table 2: Statement of Changes in Equity (₹ crore)**

| Particulars                                                                     | Balance as at April 1, 2022 | Impact on adoption of amendment to Ind AS 37# | Balance as at April 1, 2023 | Changes in equity for the year ended March 31, 2023 |
|---------------------------------------------------------------------------------|-----------------------------|-------------------------------------------------|-----------------------------|----------------------------------------------------|
| Capital reserve                                                                 | 2,098                         | —                                               | 2,098                         | —                                                |
| Capital redemption reserve                                                        | 54                           | —                                               | 54                           | —                                                |
| Securities Premium                                                              | 139                          | —                                               | 139                          | —                                                |
| Retained earnings                                                               | 61,313                        | (19)                                             | 61,294                        | 24,095                                           |
| General reserve                                                                  | 1,061                         | —                                               | 1,061                         | —                                                |
| Share Options Outstanding Account                                                 | 606                          | —                                               | 606                          | —                                                |
| Special Economic Zone Re-investment reserve (2)                                 | 8,339                         | —                                               | 8,339                         | —                                                |
| Other reserves (3)                                                              | 16                           | —                                               | 16                           | —                                                |
| Equity instruments through other comprehensive income                           | 254                          | —                                               | 254                          | (7)                                               |
| Exchange differences on translating the financial statements of a foreign operation | 1,560                         | —                                               | 1,560                         | 765                                              |
| Effective portion of Cash Flow Hedges                                            | 2                           | —                                               | 2                           | —                                                |
| Other items of other comprehensive income / (loss)                              | (292)                         | —                                               | (292)                         | 8                                                 |
| **Total**                                                                       | **75,350**                     | **(19)**                                          | **75,331**                     | **24,108**                                          |

In [ ]:
user_question = "What is the total revenue from operations for the year ending March 31, 2024?"
user_input(user_question)

The total revenue from operations for the year ending March 31, 2024 was ₹153,670 crore.


## Supporting Documents: 

**Table 1:**

| Item                           | 2024 (₹ crore) | 2023 (₹ crore) |
|---------------------------------|-----------------|-----------------|
| Unallocable expenses           | 4,678           | 4,225           |
| Other income, net              | 4,711           | 2,701           |
| Finance cost                   | 470             | 284             |
| Profit before tax              | 35,988          | 33,322          |
| Income tax expense             | 9,740           | 9,214           |
| Net Profit                     | 26,248          | 24,108          |
| Depreciation and amortization expense | 4,678           | 4,225           |
| Non-cash expenses other than depreciation and amortization | —              | —              |


**Table 2:**

| Particulars                     | Note No. | 2024 (₹ crore) | 2023 (₹ crore) | 2024 (₹ crore) | 2023 (₹ crore) |
|---------------------------------|----------|-----------------|-----------------|-----------------|-----------------|
| Revenue from operations         | 2.16     | 37,923          | 37,441          | 153,670         | 146,767         |
| Cost of Sales                  |          | 26,748          | 26,011          | 107,413         | 102,353         |
| Gross profit                    |          | 11,175          | 11,430          | 46,257          | 44,414          |
| Selling and marketing expenses |          | 1,735           | 1,659           | 6,973           | 6,249           |
| General and administration expenses |          | 1,819           | 1,894           | 7,537           | 7,260           |
| Total operating expenses       |          | 3,554           | 3,553           | 14,510          | 13,509          |
| Operating profit                |          | 7,621           | 7,877           | 31,747          | 30,905          |
| Other income, net              | 2.17     | 2,729           | 671             | 4,711           | 2,701           |
| Finance cost                   |          | 110             | 82              | 470             | 284             |
| Profit before tax              |          | 10,240          | 8,466           | 35,988          | 33,322          |
| Current tax                    | 2.15     | 1,173           | 2,260           | 8,390           | 9,287           |
| Deferred tax                   | 2.15     | 1,092           | 72              | 1,350           | (73)            |
| Profit for the period          |          | 7,975           | 6,134           | 26,248          | 24,108          |


**Table 3:**

| Items                                                                 | 2024 (₹ crore) | 2023 (₹ crore) | 2024 (₹ crore) | 2023 (₹ crore) |
|----------------------------------------------------------------------|-----------------|-----------------|-----------------|-----------------|
| Remeasurement of the net defined benefit liability/asset, net         | 26              | 25              | 120             | 8              |
| Equity instruments through other comprehensive income, net              | (12)            | (15)            | 19              | (7)             |
| Fair value changes on derivatives designated as cash flow hedge, net | 28              | 36              | 11              | (7)             |
| Exchange differences on translation of foreign operations             | (231)           | 61              | 226             | 776             |
| Fair value changes on investments, net                               | 37              | 42              | 144             | (256)           |
| Total other comprehensive income / (loss), net of tax                 | (152)           | 149             | 520             | 514             |


**Table 4:**

| Profit attributable to:          | 2024 (₹ crore) | 2023 (₹ crore) | 2024 (₹ crore) | 2023 (₹ crore) |
|-----------------------------------|-----------------|-----------------|-----------------|-----------------|
| Owners of the Company             | 7,969           | 6,128           | 26,233          | 24,095          |
| Non-controlling interests         | 6               | 6               | 15              | 13              |
| Total comprehensive income attributable to: |                 |                 |                 |                 |
| Owners of the Company             | 7,821           | 6,276           | 26,754          | 24,598          |
| Non-controlling interests         | 2               | 7               | 14              | 24              |

In [ ]:
user_question = "What was the operating profit (profit before tax minus other income) for the quarter ending March 31, 2024?"
user_input(user_question)

Here's the calculation based on the provided data:

* **Profit before tax (March 31, 2024):** ₹10,240 crore
* **Other income, net (March 31, 2024):** ₹2,729 crore

**Operating profit (March 31, 2024) = Profit before tax - Other income, net = ₹10,240 crore - ₹2,729 crore = ₹7,511 crore**


## Supporting Documents: 

**Table 1:**

| Item                               | 2024 (₹ crore) | 2023 (₹ crore) |
|------------------------------------|-----------------|-----------------|
| Unallocable expenses               | 4,678           | 4,225           |
| Other income, net                  | 4,711           | 2,701           |
| Finance cost                       | 470             | 284             |
| Profit before tax                  | 35,988          | 33,322          |
| Income tax expense                 | 9,740           | 9,214           |
| Net Profit                         | 26,248          | 24,108          |
| Depreciation and amortization expense | 4,678           | 4,225           |
| Non-cash expenses (other than D&A) | —               | —               |


**Table 2:**

| Particulars                     | Note No. | 2024 (₹ crore) | 2023 (₹ crore) | 2024 (₹ crore) | 2023 (₹ crore) |
|---------------------------------|----------|-----------------|-----------------|-----------------|-----------------|
| Revenue from operations          | 2.16     | 37,923           | 37,441           | 153,670          | 146,767          |
| Cost of Sales                   |          | 26,748           | 26,011           | 107,413          | 102,353          |
| Gross profit                     |          | 11,175           | 11,430           | 46,257           | 44,414           |
| Selling and marketing expenses   |          | 1,735            | 1,659            | 6,973            | 6,249            |
| General and administration expenses |          | 1,819            | 1,894            | 7,537            | 7,260            |
| Total operating expenses         |          | 3,554            | 3,553            | 14,510           | 13,509           |
| Operating profit                 |          | 7,621            | 7,877            | 31,747           | 30,905           |
| Other income, net               | 2.17     | 2,729            | 671             | 4,711            | 2,701            |
| Finance cost                     |          | 110              | 82              | 470              | 284             |
| Profit before tax                |          | 10,240           | 8,466            | 35,988           | 33,322           |
| Current tax                      | 2.15     | 1,173            | 2,260            | 8,390            | 9,287            |
| Deferred tax                     | 2.15     | 1,092            | 72              | 1,350            | (73)             |
| Profit for the period            |          | 7,975            | 6,134            | 26,248           | 24,108           |


**Table 3:**

| Items                                                                    | 2024 (₹ crore) | 2023 (₹ crore) | 2024 (₹ crore) | 2023 (₹ crore) |
|-------------------------------------------------------------------------|-----------------|-----------------|-----------------|-----------------|
| Remeasurement of net defined benefit liability/asset, net                | 26              | 25              | 120             | 8               |
| Equity instruments through other comprehensive income, net                 | (12)             | (15)             | 19              | (7)              |
| Fair value changes on derivatives designated as cash flow hedge, net      | 28              | 36              | 11              | (7)              |
| Exchange differences on translation of foreign operations                 | (231)            | 61              | 226             | 776             |
| Fair value changes on investments, net                                   | 37              | 42              | 144             | (256)            |
| Total other comprehensive income / (loss), net of tax                   | (152)            | 149             | 520             | 514             |
| Total comprehensive income for the period                                | 7,823            | 6,283            | 26,768           | 24,622           |
| Profit attributable to Owners of the Company                             | 7,969            | 6,128            | 26,233           | 24,095           |
| Profit attributable to Non-controlling interests                         | 6               | 6               | 15              | 13              |
| Total comprehensive income attributable to Owners of the Company          | 7,821            | 6,276            | 26,754           | 24,598           |
| Total comprehensive income attributable to Non-controlling interests     | 2               | 7               | 14              | 24              |

In [ ]:
user_question = "What is the carrying value of goodwill as of March 31, 2024?"
user_input(user_question)

The carrying value of goodwill as of March 31, 2024, was ₹7,303 crore.


## Supporting Documents: 

**Table 1: Property, Plant and Equipment - Accumulated Depreciation and Carrying Value (March 31, 2024)**

| Particulars                     | Amount (₹ crore) |
|---------------------------------|-----------------|
| Accumulated depreciation         | (18,217)        |
| Carrying value as at March 31, 2024 | 12,370          |


**Table 2: Property, Plant and Equipment - Changes in Carrying Value (Year Ended March 31, 2023)**

| Particulars                     | Land | Freehold Buildings | Plant & Machinery | Office Equipment | Computer Equipment | Furniture & Fixtures | Leasehold Improvements | Vehicles | Total |
|---------------------------------|------|--------------------|--------------------|-----------------|--------------------|----------------------|-----------------------|----------|-------|
| Gross carrying value as at April 1, 2022 | 1,431 | 11,224            | 3,210             | 1,427            | 8,527              | 2,278                 | 1,234                  | 44       | 29,375 |
| Additions - Business Combination | —    | —                 | —                 | 5                | 6                  | 1                     | 2                      | —        | 14    |
| Additions                       | 2    | 337               | 273               | 122              | 1,510              | 364                   | 220                    | 2        | 2,830 |
| Deletions*                      | (2)  | —                 | (182)             | (76)             | (1,563)            | (348)                 | (25)                   | (1)      | (2,197)|
| Translation difference          | —    | 1                  | 1                 | 4                | 39                 | 8                     | 14                     | —        | 67     |
| Gross carrying value as at March 31, 2023 | 1,431 | 11,562            | 3,302             | 1,482            | 8,519              | 2,303                 | 1,445                  | 45       | 30,089 |
| Accumulated depreciation as at April 1, 2022 | —    | (4,100)           | (2,344)           | (1,150)           | (6,034)            | (1,779)               | (856)                  | (37)     | (16,300)|
| Depreciation                    | —    | (434)             | (273)             | (121)            | (1,322)            | (236)                 | (187)                  | (4)      | (2,577)|
| Accumulated depreciation on deletions* | —    | —                 | 181               | 76                | 1,556              | 347                   | 21                     | 1        | 2,182  |
| Translation difference          | —    | (1)                | (1)               | (3)              | (26)               | (7)                    | (10)                   | —        | (48)   |
| Accumulated depreciation as at March 31, 2023 | —    | (4,535)           | (2,437)           | (1,198)           | (5,826)            | (1,675)               | (1,032)                 | (40)     | (16,743)|
| Carrying value as at April 1, 2022 | 1,431 | 7,124             | 866               | 277              | 2,493              | 499                   | 378                    | 7        | 13,075 |
| Carrying value as at March 31, 2023 | 1,431 | 7,027             | 865               | 284              | 2,693              | 628                   | 413                    | 5        | 13,346 |


**Table 3: Goodwill - Changes in Carrying Amount**

| Particulars                    | March 31, 2024 | March 31, 2023 |
|--------------------------------|-----------------|-----------------|
| Carrying value at the beginning | 7,248           | 6,195           |
| Goodwill on acquisitions       | —                | 630             |
| Translation differences        | 55               | 423             |
| Carrying value at the end      | 7,303           | 7,248           |


**Table 4: Investments (March 31, 2024 and 2023)**

*(No data provided other than the column headers)*

| Particulars             | March 31, 2024 | March 31, 2023 |
|-------------------------|-----------------|-----------------|
| Non-current Investments |                 |                 |

In [ ]:
user_question = "What was the depreciation and amortization expense for the year ending March 31, 2024?"
user_input(user_question)

The provided text gives the depreciation expense for the year ended March 31, 2024, broken down by category of property, plant, and equipment.  The total depreciation expense for the year is ₹2,685 crore.


## Supporting Documents: 

**Table 1: Changes in Carrying Value of Property, Plant and Equipment (Year Ended March 31, 2024)**

| Particulars             | Land | Freehold Buildings | Plant & Machinery | Office Equipment | Computer Equipment | Furniture & Fixtures | Leasehold Improvements | Vehicles | Total |
|--------------------------|------|--------------------|-------------------|-----------------|--------------------|----------------------|-----------------------|---------|-------|
| Gross carrying value as at April 1, 2023 | 1,431 | 11,562             | 3,302             | 1,482             | 8,519                | 2,303                  | 1,445                   | 45      | 30,089 |
| Additions                 | 1    | 300                | 193               | 106               | 931                 | 121                    | 108                    | 1       | 1,761 |
| Deletions**              | —    | (55)               | (64)              | (60)              | (846)               | (99)                   | (102)                  | (1)     | (1,227)|
| Translation difference    | —    | (37)               | (3)               | —                 | 7                   | 1                     | (4)                    | —       | (36)   |
| Gross carrying value as at March 31, 2024 | 1,432 | 11,770             | 3,428             | 1,528             | 8,611                | 2,326                  | 1,447                  | 45      | 30,587 |
| Accumulated depreciation as at April 1, 2023 | —    | (4,535)            | (2,437)            | (1,198)            | (5,826)              | (1,675)                 | (1,032)                 | (40)    | (16,743)|
| Depreciation              | —    | (450)              | (259)             | (130)             | (1,387)              | (250)                  | (206)                  | (3)     | (2,685)|
| Accumulated depreciation on deletions** | —    | 55                  | 64                 | 59                 | 836                  | 89                     | 97                     | 1       | 1,201 |
| Translation difference    | —    | 9                   | 2                  | —                  | (3)                  | (1)                    | 3                      | —       | 10     |
| Accumulated depreciation as at March 31, 2024 | —    | (4,921)            | (2,630)            | (1,269)            | (6,380)              | (1,837)                 | (1,138)                 | (42)    | (18,217)|
| Carrying value as at April 1, 2023       | 1,431 | 7,027              | 865               | 284               | 2,693                | 628                    | 413                    | 5       | 13,346 |
| Carrying value as at March 31, 2024       | 1,432 | 6,849              | 798               | 259               | 2,231                | 489                    | 309                    | 3       | 12,370 |


**Table 2: Changes in Carrying Value of Property, Plant and Equipment (Year Ended March 31, 2023)**

| Particulars             | Land | Freehold Buildings | Plant & Machinery | Office Equipment | Computer Equipment | Furniture & Fixtures | Leasehold Improvements | Vehicles | Total |
|--------------------------|------|--------------------|-------------------|-----------------|--------------------|----------------------|-----------------------|---------|-------|
| Gross carrying value as at January 1, 2023 | 1,431 | 11,530             | 3,368             | 1,466             | 8,895                | 2,450                  | 1,353                   | 44      | 30,537 |
| Additions                 | 2    | 29                 | 109               | 55                | 494                 | 162                    | 103                    | 1       | 955   |
| Deletions*               | (2)  | —                  | (175)             | (40)              | (877)               | (311)                  | (13)                   | —       | (1,418)|
| Translation difference    | —    | 3                   | —                 | 1                 | 7                   | 2                     | 2                      | —       | 15    |
| Gross carrying value as at March 31, 2023 | 1,431 | 11,562             | 3,302             | 1,482             | 8,519                | 2,303                  | 1,445                   | 45      | 30,089 |
| Accumulated depreciation as at January 1, 2023 | —    | (4,425)            | (2,547)            | (1,206)            | (6,339)              | (1,922)                 | (992)                  | (39)    | (17,470)|
| Depreciation              | —    | (109)              | (65)              | (31)              | (354)               | (62)                   | (48)                   | (1)     | (670)  |
| Accumulated depreciation on deletions* | —    | —                  | 175                | 40                 | 871                 | 310                    | 9                      | —       | 1,405 |
| Translation difference    | —    | (1)                 | —                 | (1)               | (4)                  | (1)                    | (1)                    | —       | (8)    |
| Accumulated depreciation as at March 31, 2023 | —    | (4,535)            | (2,437)            | (1,198)            | (5,826)              | (1,675)                 | (1,032)                 | (40)    | (16,743)|
| Carrying value as at January 1, 2023       | 1,431 | 7,105              | 821               | 260               | 2,556                | 528                    | 361                    | 5       | 13,067 |
| Carrying value as at March 31, 2023       | 1,431 | 7,027              | 865               | 284               | 2,693                | 628                    | 413                    | 5       | 13,346 |

In [ ]:
user_question = "How much cash and cash equivalents are reported as of March 31, 2024?"
user_input(user_question)

The cash and cash equivalents reported as of March 31, 2024, were 14,786.


## Supporting Documents: 

| Item                                      | Year Ended March 31, 2024 | Year Ended March 31, 2023 |
|-------------------------------------------|--------------------------|--------------------------|
| Other financial liabilities, other liabilities and provisions | (1,334)                     | 4,119                     |
| Cash generated from operations             | 34,441                     | 31,261                     |
| Income taxes paid                         | (9,231)                     | (8,794)                     |
| Net cash generated by operating activities | 25,210                     | 22,467                     |
| Deposits placed with corporation          | (847)                      | (996)                      |
| Redemption of deposits placed with Corporation | 710                       | 762                       |
| Payment of contingent consideration pertaining to acquisition of business | (101)                      | (60)                       |
| Escrow and other deposits pertaining to Buyback | —                         | (483)                      |
| Redemption of escrow and other deposits pertaining to Buyback | —                         | 483                       |
| Other receipts                            | 128                       | 71                        |
| Payments to acquire Investments           |                            |                          |
| Liquid mutual fund units                 | (66,191)                   | (70,631)                   |
| Target maturity fund units               | —                         | (400)                      |
| Certificates of deposit                   | (8,509)                    | (10,348)                   |
| Commercial Papers                        | (10,387)                   | (3,003)                    |
| Non-convertible debentures               | (1,526)                    | (249)                      |
| Tax free bonds and government bonds       | —                         | (27)                       |
| Government securities                    | —                         | (1,569)                    |
| Other Investments                        | (14)                       | (20)                       |
| Proceeds on sale of Investments          |                            |                          |
| Tax free bonds and government bonds       | 150                       | 221                       |
| Liquid mutual funds units                 | 64,767                    | 71,851                    |
| Certificates of deposit                   | 9,205                     | 10,404                    |
| Commercial Papers                        | 6,479                     | 2,298                     |
| Non-convertible debentures               | 1,230                     | 470                       |
| Government securities                    | 304                       | 1,882                     |
| Equity and preference securities         | 26                        | 99                        |
| Payment of lease liabilities              | (2,024)                    | (1,231)                    |
| Payment of dividends                     | (14,692)                   | (13,631)                   |
| Payment of dividend to non-controlling interest of subsidiary | (39)                       | (22)                       |
| Payment towards buyback of shares pertaining to non controlling interest of subsidiary | (18)                       | —                         |
| Shares issued on exercise of employee stock options | 5                         | 35                        |
| Other receipts                            | —                         | 132                       |
| Other payments                           | (736)                     | (479)                     |
| Buyback of equity shares including transaction cost and tax on buyback | —                         | (11,499)                   |

In [ ]:
user_question = "How much was spent on the buyback of equity shares (including transaction costs and tax) for the year ending March 31, 2023?"
user_input(user_question)

The provided text states that the buyback resulted in a cash outflow of ₹9,300 crore (excluding transaction costs and tax on buyback).  The exact amount spent including transaction costs and tax on the buyback is not available in the context.


## Supporting Documents: 

**Table 1: Dividend per Share (in ₹)**

| Particulars                     | 2024 | 2023 | 2024 | 2023 |
|---------------------------------|------|------|------|------|
| Final dividend for fiscal 2022  | —    | —    | —    | 16.00 |
| Interim dividend for fiscal 2023 | —    | —    | —    | 16.50 |
| Final dividend for fiscal 2023  | —    | —    | 17.50 | —    |
| Interim dividend for fiscal 2024 | —    | —    | 18.00 | —    |


**Table 2:  Share-Based Incentive Grants**

|                                      | \multicolumn{2}{c|}{Equity Settled RSUs}                     | \multicolumn{2}{c|}{Cash Settled RSU}                   |
|--------------------------------------|-------------------------------------------------|-------------------------------------------------|-----------------------------------------------|-----------------------------------------------|
|                                      | 2024          | 2023          | 2024          | 2023          |
|                                      | Three Months Ended March 31 | Year Ended March 31 | Three Months Ended March 31 | Year Ended March 31 |
| Key Management Personnel (KMP)       | 26,900         | 33,750         | -              | -              |
| Employees other than KMP            | 3,582,471       | 3,329,240       | -              | -              |
| **Total KMP**                       | **26,900**     | **33,750**     | **-**          | **-**          |
| **Total Employees other than KMP** | **3,582,471**   | **3,329,240**   | **-**          | **-**          |
| **Total Grants**                    | **3,609,371**   | **3,362,990**   | **-**          | **-**          |

|                                      | \multicolumn{2}{c|}{Equity Settled RSUs}                     | \multicolumn{2}{c|}{Cash Settled RSU}                   |
|--------------------------------------|-------------------------------------------------|-------------------------------------------------|-----------------------------------------------|-----------------------------------------------|
|                                      | 2024          | 2023          | 2024          | 2023          |
|                                      | Year Ended March 31 | Year Ended March 31 | Year Ended March 31 | Year Ended March 31 |
| Key Management Personnel (KMP)       | 141,171        | 210,643        | -              | -              |
| Employees other than KMP            | 4,046,731       | 3,704,014       | 169,040        | 92,400         |
| **Total KMP**                       | **141,171**     | **210,643**     | **-**          | **-**          |
| **Total Employees other than KMP** | **4,046,731**   | **3,704,014**   | **169,040**     | **92,400**     |
| **Total Grants**                    | **4,187,902**   | **3,914,657**   | **176,990**     | **92,400**     |

In [ ]:
user_question = "What is the earnings per equity share (basic) for the 3 months ending March 31, 2024?"
user_input(user_question)

The earnings per equity share (basic) for the three months ended March 31, 2024, is ₹19.25 per share.


## Supporting Documents: 

**Table 1: Income Statement (₹ crore)**

| Item                                      | Year ended March 31, 2023 | Three months ended March 31, 2024 | Year ended March 31, 2024 | Three months ended March 31, 2023 |
|-------------------------------------------|---------------------------|---------------------------------|---------------------------|---------------------------------|
| Travel expenses                           | 471                         | 426                              | 1,759                         | 1,525                              |
| Cost of software packages and others      | 3,687                       | 2,886                            | 13,515                        | 10,902                             |
| Communication expenses                     | 147                         | 171                              | 677                          | 713                               |
| Consultancy and professional charges      | 489                         | 387                              | 1,726                         | 1,684                              |
| Depreciation and amortization expenses    | 1,163                       | 1,121                             | 4,678                         | 4,225                              |
| Finance cost                              | 110                         | 82                               | 470                          | 284                               |
| Other expenses                            | 985                         | 1,146                            | 4,716                         | 4,392                              |
| **Total expenses**                        | **30,412**                   | **29,646**                         | **122,393**                   | **116,146**                         |
| Profit before tax                         | 10,240                      | 8,466                             | 35,988                      | 33,322                             |
| Current tax                               | 1,173                       | 2,260                             | 8,390                         | 9,287                              |
| Deferred tax                              | 1,092                       | 72                               | 1,350                         | (73)                               |
| **Profit for the period**                 | **7,975**                    | **6,134**                          | **26,248**                    | **24,108**                          |
| Remeasurement of the net defined benefit liability/asset, net | 26                          | 25                               | 120                          | 8                                 |
| Equity instruments through other comprehensive income, net | (12)                         | (15)                             | 19                           | (7)                                |
| Fair value changes on derivatives designated as cash flow hedge, net | 28                          | 36                               | 11                           | (7)                                |
| Fair value changes on investments, net    | 37                          | 42                               | 144                          | (256)                              |
| **Total other comprehensive income /(loss), net of tax** | **(152)**                   | **149**                            | **520**                        | **514**                             |
| **Total comprehensive income for the period** | **7,823**                    | **6,283**                          | **26,768**                    | **24,622**                          |
| Profit attributable to Owners of the Company | 7,969                       | 6,128                             | 26,233                       | 24,095                             |
| Profit attributable to Non-controlling interests | 6                           | 6                               | 15                           | 13                                |
| **Total comprehensive income attributable to Owners of the Company** | **7,821**                    | **6,276**                          | **26,754**                    | **24,598**                          |
| **Total comprehensive income attributable to Non-controlling interests** | **2**                         | **7**                              | **14**                         | **24**                              |
| Basic (in ₹ per share)                   | 19.25                       | 14.79                             | 63.39                         | 57.63                              |
| Diluted (in ₹ per share)                 | 19.22                       | 14.77                             | 63.29                         | 57.54                              |


**Table 2: Statement of Changes in Equity (₹ crore)**

| Particulars                                                                     | Balance as at April 1, 2022 | Impact on adoption of amendment to Ind AS 37# | Balance as at April 1, 2023 | Changes in equity for the year ended March 31, 2023 |
|---------------------------------------------------------------------------------|-----------------------------|-------------------------------------------------|-----------------------------|------------------------------------------------------|
| Capital reserve                                                                 | 2,098                         | —                                                | 2,098                         | —                                                    |
| Capital redemption reserve                                                        | 54                          | —                                                | 54                          | —                                                    |
| Securities Premium                                                              | 139                         | —                                                | 139                         | —                                                    |
| Retained earnings                                                                | 61,313                       | (19)                                              | 61,294                       | 24,095                                               |
| General reserve                                                                  | 1,061                        | —                                                | 1,061                        | —                                                    |
| Share Options Outstanding Account                                                 | 606                         | —                                                | 606                         | —                                                    |
| Special Economic Zone Re-investment reserve (2)                                 | 8,339                        | —                                                | 8,339                        | —                                                    |
| Other reserves (3)                                                              | 16                          | —                                                | 16                          | —                                                    |
| Equity instruments through other comprehensive income                           | 254                         | —                                                | 254                         | (7)                                                 |
| Exchange differences on translating the financial statements of a foreign operation | 1,560                        | —                                                | 1,560                        | 765                                                |
| Effective portion of Cash Flow Hedges                                            | 2                           | —                                                | 2                           | —                                                    |
| Other items of other comprehensive income / (loss)                              | (292)                        | —                                                | (292)                        | 8                                                   |
| **Total Equity**                                                                | **75,350**                    | **(19)**                                           | **75,331**                    | **24,108**                                              |